In [165]:
from selenium import webdriver
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By

# Set up the Selenium WebDriver (make sure to have the appropriate webdriver installed)
driver = webdriver.Edge(executable_path=r"C:\Program Files (x86)\msedgedriver.exe")

# URL of the webpage containing the dropdown menu
url = "https://www.nba.com/stats/leaders?SeasonType=Regular+Season&Season=2023-24"
driver.get(url)

In [170]:
years = driver.find_elements_by_class_name("DropDown_select__4pIg9")[0].text.split('\n')


table_data = []
for i in range(len(years[:2])):
    year = years[i]    
    
    season_dropdown_menu = Select(driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[3]/section[1]/div/div/div[1]/label/div/select')) 
    season_dropdown_menu.select_by_visible_text(year)
    
    time.sleep(5)
    # Wait for the page dropdown to be present
    page_dropdown = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select'))
    )

    # Find the page dropdown element using its HTML attribute class name
    page_dropdown = Select(driver.find_elements_by_class_name("DropDown_select__4pIg9")[5])

    # Change the selected option by index, value 'All'
    page_dropdown.select_by_visible_text('All')
    
    table = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table'))
    )
    
    # Store the table data in a variable for looping
    tbody = table.find_element(By.TAG_NAME, 'tbody')
    rows = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.TAG_NAME, 'tr'))
    )
    
    for row in rows:
        try:
            row_cells = row.find_elements(By.TAG_NAME, 'td')
            row_text = [cell.text for cell in row_cells]
            row_text.append(year)
            table_data.append(row_text) 
            print(f"Adding data for player #{row_text[0]} {row_text[1]} for the {year} season")
        except:
            # Handle StaleElementReferenceException by refinding the row
            rows = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.TAG_NAME, 'tr'))
            )

Adding data for player #1 Joel Embiid for the 2023-24 season
Adding data for player #2 Stephen Curry for the 2023-24 season
Adding data for player #3 Luka Doncic for the 2023-24 season
Adding data for player #4 Kevin Durant for the 2023-24 season
Adding data for player #5 Nikola Jokic for the 2023-24 season
Adding data for player #6 Giannis Antetokounmpo for the 2023-24 season
Adding data for player #7 Shai Gilgeous-Alexander for the 2023-24 season
Adding data for player #8 Donovan Mitchell for the 2023-24 season
Adding data for player #9 Jayson Tatum for the 2023-24 season
Adding data for player #10 Tyrese Maxey for the 2023-24 season
Adding data for player #11 Anthony Edwards for the 2023-24 season
Adding data for player #12 Desmond Bane for the 2023-24 season
Adding data for player #13 Paul George for the 2023-24 season
Adding data for player #13 Tyrese Haliburton for the 2023-24 season
Adding data for player #15 LeBron James for the 2023-24 season
Adding data for player #16 Damian 

In [169]:
# Find the page dropdown element using its HTML attribute class name
page_dropdown = Select(driver.find_elements_by_class_name("DropDown_select__4pIg9")[5])

# Change the selected option by index, value 'All'
page_dropdown.select_by_visible_text('All')

In [ ]:
# Get table headers
headers = table.find_element_by_xpath('//thead').text.split(' ')
headers.append('YEAR')

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: MicrosoftEdge=119.0.2151.58)


In [ ]:
import pandas as pd

# Getting the headers from the data array
# It is important to remove them from the data array afterwards in order to be parsed correctly by Pandas
df = pd.DataFrame(table_data, columns=headers)
df


,#,PLAYER,TEAM,GP,MIN,PTS,FGM,FGA,FG%,3PM,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,EFF,YEAR
0,2023-24,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,1,Joel Embiid,PHI,11,34.1,31.9,10.7,21.3,50.4,1.1,...,87.3,3.4,8.1,11.5,5.8,0.9,1.7,3.7,36.2,2023-24
2,2,Stephen Curry,GSW,11,32.5,30.7,9.5,19.5,48.8,5.6,...,93.0,0.9,3.7,4.6,3.9,1.0,0.2,3.7,26.3,2023-24
3,3,Luka Doncic,DAL,12,34.8,30.7,10.3,20.6,50.2,4.2,...,76.1,0.7,7.1,7.8,8.1,1.1,0.4,4.4,31.5,2023-24
4,4,Kevin Durant,PHX,11,36.6,30.1,10.2,19.8,51.4,1.8,...,86.1,0.5,6.5,7.0,4.7,0.8,1.2,3.9,29.0,2023-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,268,Jalen McDaniels,TOR,9,9.8,1.3,0.6,3.0,18.5,0.2,...,0.0,0.1,1.2,1.3,0.6,0.3,0.2,0.2,1.1,2023-24
267,269,Cody Zeller,NOP,8,6.6,1.0,0.3,1.3,20.0,0.0,...,40.0,0.9,1.6,2.5,1.0,0.1,0.0,0.3,2.6,2023-24
268,270,Kessler Edwards,SAC,8,4.5,0.5,0.1,0.8,16.7,0.1,...,50.0,0.0,0.6,0.6,0.4,0.1,0.1,0.1,0.9,2023-24
269,2022-23,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271 entries, 0 to 270
Data columns (total 24 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   #       271 non-null    object
 1   PLAYER  269 non-null    object
 2   TEAM    269 non-null    object
 3   GP      269 non-null    object
 4   MIN     269 non-null    object
 5   PTS     269 non-null    object
 6   FGM     269 non-null    object
 7   FGA     269 non-null    object
 8   FG%     269 non-null    object
 9   3PM     269 non-null    object
 10  3PA     269 non-null    object
 11  3P%     269 non-null    object
 12  FTM     269 non-null    object
 13  FTA     269 non-null    object
 14  FT%     269 non-null    object
 15  OREB    269 non-null    object
 16  DREB    269 non-null    object
 17  REB     269 non-null    object
 18  AST     269 non-null    object
 19  STL     269 non-null    object
 20  BLK     269 non-null    object
 21  TOV     269 non-null    object
 22  EFF     269 non-null    ob